In [1]:
import pandas as pd
import numpy as np

# Разбивка по годам 

## Знакомство с данными

In [18]:
df_orders = pd.read_csv('orders.csv')
df_orders['order_month'] = df_orders['order_date'].apply(lambda x: x[0:7])
df_orders['order_year'] = df_orders['order_date'].apply(lambda x: x[0:4])
df_orders.head()

,id,order_date,ship_mode,customer_id,sales,order_month,order_year
0,100006,2014-09-07,Standard,DK-13375,377.970,2014-09,2014
1,100090,2014-07-08,Standard,EB-13705,699.192,2014-07,2014
2,100293,2014-03-14,Standard,NF-18475,91.056,2014-03,2014
3,100328,2014-01-28,Standard,JC-15340,3.928,2014-01,2014
4,100363,2014-04-08,Standard,JM-15655,21.376,2014-04,2014


In [19]:
df_customers = pd.read_csv('customers.csv')
df_customers.rename(columns = {'id':'customer_id'}, inplace=True)
df_customers.head()

,customer_id,name,segment,state,city
0,CG-12520,Claire Gute,Consumer,Kentucky,Henderson
1,DV-13045,Darrin Van Huff,Corporate,California,Los Angeles
2,SO-20335,Sean O'Donnell,Consumer,Florida,Fort Lauderdale
3,BH-11710,Brosina Hoffman,Consumer,California,Los Angeles
4,AA-10480,Andrew Allen,Consumer,North Carolina,Concord


In [26]:
df_merge = pd.merge(df_orders, df_customers, 'inner', on='customer_id')
df_merge.sample(5)

,id,order_date,ship_mode,customer_id,sales,order_month,order_year,name,segment,state,city
4057,137127,2016-06-16,Standard,JJ-15445,9.450,2016-06,2016,Jennifer Jackson,Consumer,California,Los Angeles
2817,123834,2017-07-21,Standard,GM-14500,124.404,2017-07,2017,Gene McClure,Consumer,Rhode Island,Providence
4217,132486,2015-10-23,Second,JF-15355,1446.784,2015-10,2015,Jay Fein,Consumer,Texas,Austin
2142,163629,2017-11-17,Standard,RA-19885,286.090,2017-11,2017,Ruben Ausman,Corporate,California,Los Angeles
4794,147032,2017-07-31,Standard,LB-16795,872.500,2017-07,2017,Laurel Beltran,Home Office,Illinois,Highland Park


In [27]:
df_first = df_merge.groupby('customer_id')['order_year'].min().reset_index()
df_first.columns = ['customer_id', 'first_order']
df_first.sample(5)

,customer_id,first_order
422,KD-16270,2014
732,TB-21175,2014
394,JM-15865,2014
440,KN-16450,2014
410,JW-15220,2015


In [28]:
df_full = pd.merge(df_merge, df_first, 'inner', on='customer_id')
df_full.sample(5)

,id,order_date,ship_mode,customer_id,sales,order_month,order_year,name,segment,state,city,first_order
458,107755,2014-02-07,Standard,CK-12760,115.360,2014-02,2014,Cyma Kinney,Corporate,New Jersey,Linden,2014
4866,136021,2016-08-14,Standard,JM-15580,121.092,2016-08,2016,Jill Matthias,Consumer,North Carolina,Monroe,2016
1057,112326,2014-01-04,Standard,PO-19195,288.060,2014-01,2014,Phillina Ober,Home Office,Illinois,Naperville,2014
2072,103674,2016-12-06,Standard,AP-10720,1018.302,2016-12,2016,Anne Pryor,Home Office,New York,New York City,2014
2008,134607,2017-10-03,Second,HL-15040,22.580,2017-10,2017,Hunter Lopez,Consumer,Texas,Houston,2014


In [54]:
df_users = df_full.groupby(['customer_id', 'segment', 'ship_mode', 'first_order', 'order_year'])\
    .agg({'sales':'sum', 'id':'count'}).reset_index()
df_users.rename(columns={'sales': 'sales_sum', 'id': 'order_counts'}, inplace=True)
df_users.head(5)

,customer_id,segment,ship_mode,first_order,order_year,sales_sum,order_counts
0,AA-10315,Consumer,First,2014,2015,26.960,1
1,AA-10315,Consumer,Standard,2014,2014,756.048,2
2,AA-10315,Consumer,Standard,2014,2016,4406.072,1
3,AA-10315,Consumer,Standard,2014,2017,374.480,1
4,AA-10375,Consumer,First,2014,2015,178.370,1


In [55]:
def get_crosstab(df, value_column):
    df_crosstab = pd.crosstab(df.customer_id, df.order_year, values=df[value_column], aggfunc=sum)\
        .reset_index()
    df_crosstab = pd.merge(df_crosstab, df_first, how='inner', on='customer_id')
    del df_crosstab['customer_id']
    return df_crosstab

In [56]:
df_test = get_crosstab(df_users, 'order_counts')

In [57]:
df_test.head(5)

,2014,2015,2016,2017,first_order
0,2.0,1.0,1.0,1.0,2014
1,2.0,3.0,2.0,2.0,2014
2,1.0,NaN,2.0,1.0,2014
3,2.0,1.0,2.0,1.0,2014
4,2.0,NaN,1.0,NaN,2014


In [58]:
def get_retention(df):
    df_triangle = df.groupby(['first_order']).sum().reset_index()
    df_triangle['Base'] = np.diag(df_triangle.iloc[:,1:])
    df_retention = pd.DataFrame(df_triangle[['first_order', 'Base']])
    df_retention.columns = ['Cohort', 'Base']

    for col in df_triangle.columns[1:-1]:
        df_retention[col] = round(df_triangle[col] / df_triangle['Base'], 3)

    for i in range(2, len(df_retention.columns)):
        df_retention.iloc[i-1:, i] = None
        
    return df_retention

## Выводы

### Sales retention

In [59]:
get_retention(get_crosstab(df_users, 'sales_sum'))

,Cohort,Base,2014,2015,2016,2017
0,2014,484247.4981,1.0,0.754,0.939,1.101
1,2015,105388.4962,NaN,1.000,0.944,1.250
2,2016,54803.6200,NaN,NaN,1.000,1.111
3,2017,7511.8050,NaN,NaN,NaN,1.000


### Transaction retention

In [60]:
get_retention(get_crosstab(df_users, 'order_counts'))

,Cohort,Base,2014,2015,2016,2017
0,2014,969.0,1.0,0.833,1.028,1.288
1,2015,231.0,NaN,1.000,0.931,1.268
2,2016,104.0,NaN,NaN,1.000,1.221
3,2017,19.0,NaN,NaN,NaN,1.000


### Sales retention by segment

In [62]:
for segment in df_users.segment.unique():
    print(f'\nSegment: {segment}')
    print(get_retention(get_crosstab(df_users[df_users.segment==segment], 'sales_sum')))


Segment: Consumer
  Cohort         Base  2014   2015   2016   2017
0   2014  266096.8126   1.0  0.789  0.818  0.906
1   2015   56590.8551   NaN  1.000  0.832  0.817
2   2016   32119.4550   NaN    NaN  1.000  1.229
3   2017    4985.0760   NaN    NaN    NaN  1.000

Segment: Home Office
  Cohort        Base  2014   2015   2016   2017
0   2014  89715.8118   1.0  0.616  0.874  1.296
1   2015  19942.1216   NaN  1.000  1.120  1.475
2   2016   4483.5420   NaN    NaN  1.000  2.905
3   2017    729.6480   NaN    NaN    NaN  1.000

Segment: Corporate
  Cohort         Base  2014   2015   2016   2017
0   2014  128434.8737   1.0  0.778  1.237  1.367
1   2015   28855.5195   NaN  1.000  1.043  1.944
2   2016   18200.6230   NaN    NaN  1.000  0.462
3   2017    1797.0810   NaN    NaN    NaN  1.000


### Sales retention by ship_mode

In [63]:
for ship_mode in df_users.ship_mode.unique():
    print(f'\nShip_mode: {ship_mode}')
    print(get_retention(get_crosstab(df_users[df_users.ship_mode==ship_mode], 'sales_sum')))


Ship_mode: First
  Cohort        Base  2014   2015   2016   2017
0   2014  59769.2645   1.0  0.997  1.010  1.590
1   2015   9644.0606   NaN  1.000  1.632  2.143
2   2016   6153.2850   NaN    NaN  1.000  3.814
3   2017    955.7840   NaN    NaN    NaN  1.000

Ship_mode: Standard
  Cohort         Base  2014   2015   2016   2017
0   2014  305621.3196   1.0  0.716  0.921  0.935
1   2015   65723.8550   NaN  1.000  0.791  1.142
2   2016   38855.4240   NaN    NaN  1.000  0.752
3   2017    5644.8960   NaN    NaN    NaN  1.000

Ship_mode: Second
  Cohort         Base  2014   2015   2016   2017
0   2014  101386.7800   1.0  0.642  0.811  1.105
1   2015   23970.2056   NaN  1.000  1.180  1.216
2   2016    9537.8710   NaN    NaN  1.000  0.697
3   2017     911.1250   NaN    NaN    NaN  1.000

Ship_mode: Same Day
  Cohort       Base  2014   2015   2016   2017
0   2014  17470.134   1.0  1.234  1.759  2.308
1   2015   6050.375   NaN  1.000  0.581  1.136
2   2016    257.040   NaN    NaN  1.000  6.163


### Transaction retention by segment

In [64]:
for segment in df_users.segment.unique():
    print(f'\nSegment: {segment}')
    print(get_retention(get_crosstab(df_users[df_users.segment==segment], 'order_counts')))


Segment: Consumer
  Cohort   Base  2014   2015   2016   2017
0   2014  524.0   1.0  0.817  0.954  1.265
1   2015  107.0   NaN  1.000  0.869  1.215
2   2016   58.0   NaN    NaN  1.000  1.207
3   2017   13.0   NaN    NaN    NaN  1.000

Segment: Home Office
  Cohort   Base  2014   2015   2016   2017
0   2014  158.0   1.0  0.899  1.095  1.405
1   2015   49.0   NaN  1.000  1.122  1.429
2   2016   14.0   NaN    NaN  1.000  1.786
3   2017    1.0   NaN    NaN    NaN  1.000

Segment: Corporate
  Cohort   Base  2014   2015   2016   2017
0   2014  287.0   1.0  0.826  1.125  1.265
1   2015   75.0   NaN  1.000  0.893  1.240
2   2016   32.0   NaN    NaN  1.000  1.000
3   2017    5.0   NaN    NaN    NaN  1.000


### Transaction retention by ship_mode

In [65]:
for ship_mode in df_users.ship_mode.unique():
    print(f'\nShip_mode: {ship_mode}')
    print(get_retention(get_crosstab(df_users[df_users.ship_mode==ship_mode], 'order_counts')))


Ship_mode: First
  Cohort   Base  2014   2015   2016   2017
0   2014  145.0   1.0  0.772  1.103  1.448
1   2015   31.0   NaN  1.000  1.194  1.516
2   2016   18.0   NaN    NaN  1.000  1.278
3   2017    4.0   NaN    NaN    NaN  1.000

Ship_mode: Standard
  Cohort   Base  2014   2015   2016   2017
0   2014  586.0   1.0  0.845  1.002  1.229
1   2015  141.0   NaN  1.000  0.943  1.284
2   2016   62.0   NaN    NaN  1.000  1.242
3   2017   12.0   NaN    NaN    NaN  1.000

Ship_mode: Second
  Cohort   Base  2014   2015   2016   2017
0   2014  190.0   1.0  0.826  0.995  1.311
1   2015   49.0   NaN  1.000  0.673  1.020
2   2016   22.0   NaN    NaN  1.000  1.000
3   2017    3.0   NaN    NaN    NaN  1.000

Ship_mode: Same Day
  Cohort  Base  2014   2015  2016   2017
0   2014  48.0   1.0  0.896  1.25  1.438
1   2015  10.0   NaN  1.000  1.20  1.500
2   2016   2.0   NaN    NaN  1.00  2.500


## Предподготовленный ответ из ноутбука
#### 1. Тотально sales и transaction ретеншены 2016 лучше 2015
#### 2. Но есть слабые когорты, где хуже: 
#### - Для sales-retention это:
    - Segment: Consumer, first order=2015
    - Ship_mode: Standard, first order=2015
#### - Для transaction-retention это:
    - Segment: Consumer, first order=2015
    - Segment: Corporate, first order=2015
    - Ship_mode: Standard, first order=2015
    - Ship_mode: Second, first order=2015


# Разбивка по месяцам

Уточнение контекста:
- С одной стороны в первоначальном ответе приводятся выводы относительно 2015 и 2016 года
- С другой стороны в условии сказано "Он просит вас сравнить, на сколько сейчас все хуже, чем в прошлом году?"
- В датасете последняя информация относится к 2017 году, значит на дворе сейчас как минимум 2017 год и сравнивать нужно 2017 и 2016
- А раз сравнивать нужно последний год, то с разбивкой по году делать выоды о ретеншне не получится, нужно считать по месяцам


In [91]:
df_orders['order_month'].value_counts().sort_index()

2014-01     32
2014-02     28
2014-03     71
2014-04     66
2014-05     69
2014-06     66
2014-07     65
2014-08     72
2014-09    130
2014-10     78
2014-11    151
2014-12    141
2015-01     29
2015-02     36
2015-03     79
2015-04     72
2015-05     74
2015-06     68
2015-07     66
2015-08     68
2015-09    140
2015-10     87
2015-11    158
2015-12    161
2016-01     48
2016-02     45
2016-03     86
2016-04     89
2016-05    108
2016-06     97
2016-07     96
2016-08     90
2016-09    192
2016-10    105
2016-11    183
2016-12    176
2017-01     69
2017-02     53
2017-03    118
2017-04    116
2017-05    118
2017-06    133
2017-07    111
2017-08    111
2017-09    226
2017-10    147
2017-11    261
2017-12    224
Name: order_month, dtype: int64

In [92]:
df_merge = pd.merge(df_orders, df_customers, 'inner', on='customer_id')
df_merge.sample(5)
assert df_merge.shape[0] == df_orders.shape[0]

In [95]:
df_first = df_merge.groupby('customer_id')['order_month'].min().reset_index()
df_first.columns = ['customer_id', 'first_order']
df_first

,customer_id,first_order
0,AA-10315,2014-03
1,AA-10375,2014-04
2,AA-10480,2014-05
3,AA-10645,2014-06
4,AB-10015,2014-02
...,...,...
788,XP-21865,2014-01
789,YC-21895,2014-11
790,YS-21880,2015-01
791,ZC-21910,2014-10


In [96]:
df_full = pd.merge(df_merge, df_first, 'inner', on='customer_id')
df_full

,id,order_date,ship_mode,customer_id,sales,order_month,order_year,name,segment,state,city,first_order
0,100006,2014-09-07,Standard,DK-13375,377.970,2014-09,2014,Dennis Kane,Consumer,Ohio,Marion,2014-09
1,131884,2015-12-06,Same Day,DK-13375,594.002,2015-12,2015,Dennis Kane,Consumer,Ohio,Marion,2014-09
2,145065,2015-12-12,First,DK-13375,32.308,2015-12,2015,Dennis Kane,Consumer,Ohio,Marion,2014-09
3,133046,2017-07-27,Second,DK-13375,297.990,2017-07,2017,Dennis Kane,Consumer,Ohio,Marion,2014-09
4,165099,2017-12-11,First,DK-13375,1.392,2017-12,2017,Dennis Kane,Consumer,Ohio,Marion,2014-09
...,...,...,...,...,...,...,...,...,...,...,...,...
5004,168193,2017-03-06,Second,RM-19750,98.350,2017-03,2017,Roland Murray,Consumer,New York,New York City,2017-03
5005,122140,2015-04-02,Standard,MO-17950,130.288,2015-04,2015,Michael Oakman,Consumer,Texas,Dallas,2015-04
5006,166233,2017-07-03,Standard,MO-17950,24.000,2017-07,2017,Michael Oakman,Consumer,Texas,Dallas,2015-04
5007,160150,2015-07-19,First,TS-21085,2.025,2015-07,2015,Thais Sissman,Consumer,Arizona,Phoenix,2015-07


In [104]:
df_users = df_full.groupby(['customer_id', 'segment', 'ship_mode', 'first_order', 'order_month'])\
    .agg({'sales':'sum', 'id':'count'}).reset_index()
df_users.rename(columns={'sales': 'sales_sum', 'id': 'order_counts'}, inplace=True)
df_users.head(5)

,customer_id,segment,ship_mode,first_order,order_month,sales_sum,order_counts
0,AA-10315,Consumer,First,2014-03,2015-10,26.960,1
1,AA-10315,Consumer,Standard,2014-03,2014-03,726.548,1
2,AA-10315,Consumer,Standard,2014-03,2014-09,29.500,1
3,AA-10315,Consumer,Standard,2014-03,2016-03,4406.072,1
4,AA-10315,Consumer,Standard,2014-03,2017-06,374.480,1


In [105]:
def get_crosstab(df, value_column):
    df_crosstab = pd.crosstab(df['customer_id'], df['order_month'], values=df[value_column], aggfunc=sum).reset_index()
    df_crosstab = pd.merge(df_crosstab, df_first, how='inner', on='customer_id')
    del df_crosstab['customer_id']
    return df_crosstab

In [106]:
df_test = get_crosstab(df_users, 'order_counts')

In [107]:
df_test.head(5)

,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,...,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,first_order
0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2014-03
1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,2014-04
2,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-05
3,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2014-06
4,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-02


In [101]:
def get_retention(df):
    df_triangle = df.groupby(['first_order']).sum().reset_index()
    df_triangle['Base'] = np.diag(df_triangle.iloc[:,1:])
    df_retention = pd.DataFrame(df_triangle[['first_order', 'Base']])
    df_retention.columns = ['Cohort', 'Base']

    for col in df_triangle.columns[1:-1]:
        df_retention[col] = round(df_triangle[col] / df_triangle['Base'], 3)

    for i in range(2, len(df_retention.columns)):
        df_retention.iloc[i-1:, i] = None
        
    return df_retention

In [122]:
df = get_crosstab(df_users, 'sales_sum')

In [123]:
df_triangle = df.groupby(['first_order']).sum().reset_index()
df_triangle

,first_order,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,...,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12
0,2014-01,14236.895,225.232,0.000,1472.382,47.750,0.0000,116.298,907.5840,1609.7000,...,2788.5840,204.0960,1576.3090,1960.7000,807.898,357.080,8075.530,2051.5690,12298.7620,665.1200
1,2014-02,0.000,4294.660,634.310,596.252,576.234,0.0000,697.386,1040.4000,11907.5250,...,882.2720,560.6460,159.2800,6447.9050,782.616,1072.896,996.902,1571.1650,3605.5620,604.5340
2,2014-03,0.000,0.000,55056.699,1519.546,2543.338,3298.7236,3101.218,0.0000,2860.1240,...,3862.4560,3071.1980,4769.0484,5709.8240,3162.986,3007.568,4786.222,2486.5112,8718.4640,6102.4180
3,2014-04,0.000,0.000,0.000,24707.165,2410.624,875.4550,2325.468,454.1520,3195.5730,...,5106.9908,245.8160,5813.0308,2420.3280,5826.792,4147.648,6109.317,3210.6740,4051.6612,7996.4180
4,2014-05,0.000,0.000,0.000,0.000,18070.341,1685.1280,1862.666,653.2600,8717.3390,...,133.3740,1059.8400,2136.9020,4750.6640,2512.434,677.482,3327.961,5503.2600,11595.3190,2290.7400
5,2014-06,0.000,0.000,0.000,0.000,0.000,28735.8210,351.216,31.0000,733.9520,...,3516.6800,4480.6740,2886.1700,3690.6780,2263.940,285.380,6760.413,1804.0200,15355.6300,3699.5160
6,2014-07,0.000,0.000,0.000,0.000,0.000,0.0000,25492.141,2188.3790,1044.6610,...,1000.8400,6748.2260,3643.1280,4464.9212,881.530,4970.386,2408.282,7954.1930,5918.6200,5072.9750
7,2014-08,0.000,0.000,0.000,0.000,0.000,0.0000,0.000,22634.6935,8059.5820,...,960.8720,891.2220,2631.8960,2778.4200,2924.816,2435.680,1719.230,3803.0120,4280.2870,8030.0250
8,2014-09,0.000,0.000,0.000,0.000,0.000,0.0000,0.000,0.0000,43648.8948,...,2186.3240,4929.8765,1218.9870,3730.2160,9335.470,3680.098,16926.116,18347.6520,4262.4210,4303.8500
9,2014-10,0.000,0.000,0.000,0.000,0.000,0.0000,0.000,0.0000,0.0000,...,4493.1900,620.8960,744.0030,4433.4180,2649.332,1815.560,3323.007,8607.8940,5794.7840,3558.1670


In [125]:
df_triangle['Base'] = np.diag(df_triangle.iloc[:,1:])
df_triangle

,first_order,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,...,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,Base
0,2014-01,14236.895,225.232,0.000,1472.382,47.750,0.0000,116.298,907.5840,1609.7000,...,204.0960,1576.3090,1960.7000,807.898,357.080,8075.530,2051.5690,12298.7620,665.1200,14236.8950
1,2014-02,0.000,4294.660,634.310,596.252,576.234,0.0000,697.386,1040.4000,11907.5250,...,560.6460,159.2800,6447.9050,782.616,1072.896,996.902,1571.1650,3605.5620,604.5340,4294.6600
2,2014-03,0.000,0.000,55056.699,1519.546,2543.338,3298.7236,3101.218,0.0000,2860.1240,...,3071.1980,4769.0484,5709.8240,3162.986,3007.568,4786.222,2486.5112,8718.4640,6102.4180,55056.6990
3,2014-04,0.000,0.000,0.000,24707.165,2410.624,875.4550,2325.468,454.1520,3195.5730,...,245.8160,5813.0308,2420.3280,5826.792,4147.648,6109.317,3210.6740,4051.6612,7996.4180,24707.1650
4,2014-05,0.000,0.000,0.000,0.000,18070.341,1685.1280,1862.666,653.2600,8717.3390,...,1059.8400,2136.9020,4750.6640,2512.434,677.482,3327.961,5503.2600,11595.3190,2290.7400,18070.3410
5,2014-06,0.000,0.000,0.000,0.000,0.000,28735.8210,351.216,31.0000,733.9520,...,4480.6740,2886.1700,3690.6780,2263.940,285.380,6760.413,1804.0200,15355.6300,3699.5160,28735.8210
6,2014-07,0.000,0.000,0.000,0.000,0.000,0.0000,25492.141,2188.3790,1044.6610,...,6748.2260,3643.1280,4464.9212,881.530,4970.386,2408.282,7954.1930,5918.6200,5072.9750,25492.1410
7,2014-08,0.000,0.000,0.000,0.000,0.000,0.0000,0.000,22634.6935,8059.5820,...,891.2220,2631.8960,2778.4200,2924.816,2435.680,1719.230,3803.0120,4280.2870,8030.0250,22634.6935
8,2014-09,0.000,0.000,0.000,0.000,0.000,0.0000,0.000,0.0000,43648.8948,...,4929.8765,1218.9870,3730.2160,9335.470,3680.098,16926.116,18347.6520,4262.4210,4303.8500,43648.8948
9,2014-10,0.000,0.000,0.000,0.000,0.000,0.0000,0.000,0.0000,0.0000,...,620.8960,744.0030,4433.4180,2649.332,1815.560,3323.007,8607.8940,5794.7840,3558.1670,20829.9950


In [118]:
get_retention()

,Cohort,Base,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,...,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12
0,2014-01,14236.8950,1.0,0.016,0.000,0.103,0.003,0.000,0.008,0.064,...,0.196,0.014,0.111,0.138,0.057,0.025,0.567,0.144,0.864,0.047
1,2014-02,4294.6600,NaN,1.000,0.148,0.139,0.134,0.000,0.162,0.242,...,0.205,0.131,0.037,1.501,0.182,0.250,0.232,0.366,0.840,0.141
2,2014-03,55056.6990,NaN,NaN,1.000,0.028,0.046,0.060,0.056,0.000,...,0.070,0.056,0.087,0.104,0.057,0.055,0.087,0.045,0.158,0.111
3,2014-04,24707.1650,NaN,NaN,NaN,1.000,0.098,0.035,0.094,0.018,...,0.207,0.010,0.235,0.098,0.236,0.168,0.247,0.130,0.164,0.324
4,2014-05,18070.3410,NaN,NaN,NaN,NaN,1.000,0.093,0.103,0.036,...,0.007,0.059,0.118,0.263,0.139,0.037,0.184,0.305,0.642,0.127
5,2014-06,28735.8210,NaN,NaN,NaN,NaN,NaN,1.000,0.012,0.001,...,0.122,0.156,0.100,0.128,0.079,0.010,0.235,0.063,0.534,0.129
6,2014-07,25492.1410,NaN,NaN,NaN,NaN,NaN,NaN,1.000,0.086,...,0.039,0.265,0.143,0.175,0.035,0.195,0.094,0.312,0.232,0.199
7,2014-08,22634.6935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,...,0.042,0.039,0.116,0.123,0.129,0.108,0.076,0.168,0.189,0.355
8,2014-09,43648.8948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.050,0.113,0.028,0.085,0.214,0.084,0.388,0.420,0.098,0.099
9,2014-10,20829.9950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.216,0.030,0.036,0.213,0.127,0.087,0.160,0.413,0.278,0.171


In [119]:
get_crosstab(df_users, 'sales_sum')

,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,...,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,first_order
0,NaN,NaN,726.548,NaN,NaN,NaN,NaN,NaN,29.500,NaN,...,NaN,NaN,374.480,NaN,NaN,NaN,NaN,NaN,NaN,2014-03
1,NaN,NaN,NaN,16.52,NaN,NaN,NaN,NaN,NaN,34.272,...,NaN,NaN,NaN,NaN,NaN,56.86,NaN,NaN,149.872,2014-04
2,NaN,NaN,NaN,NaN,27.46,NaN,NaN,NaN,NaN,NaN,...,15.552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-05
3,NaN,NaN,NaN,NaN,NaN,1106.77,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.960,NaN,2014-06
4,NaN,12.624,309.592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,708.616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,337.088,9.090,...,NaN,NaN,NaN,NaN,NaN,NaN,185.38,263.932,NaN,2014-01
789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750.680,2014-11
790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,436.704,34.50,NaN,9.552,3859.488,NaN,NaN,NaN,1000.020,2015-01
791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1549.770,...,NaN,43.92,153.456,NaN,NaN,NaN,NaN,29.690,NaN,2014-10


In [121]:
get_crosstab(df_users, 'sales_sum')

,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,...,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,first_order
0,NaN,NaN,726.548,NaN,NaN,NaN,NaN,NaN,29.500,NaN,...,NaN,NaN,374.480,NaN,NaN,NaN,NaN,NaN,NaN,2014-03
1,NaN,NaN,NaN,16.52,NaN,NaN,NaN,NaN,NaN,34.272,...,NaN,NaN,NaN,NaN,NaN,56.86,NaN,NaN,149.872,2014-04
2,NaN,NaN,NaN,NaN,27.46,NaN,NaN,NaN,NaN,NaN,...,15.552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-05
3,NaN,NaN,NaN,NaN,NaN,1106.77,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.960,NaN,2014-06
4,NaN,12.624,309.592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,708.616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,337.088,9.090,...,NaN,NaN,NaN,NaN,NaN,NaN,185.38,263.932,NaN,2014-01
789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750.680,2014-11
790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,436.704,34.50,NaN,9.552,3859.488,NaN,NaN,NaN,1000.020,2015-01
791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1549.770,...,NaN,43.92,153.456,NaN,NaN,NaN,NaN,29.690,NaN,2014-10


## Выводы

### Sales retention

In [ ]:
df_triangle = df.groupby(['first_order']).sum().reset_index()
df_triangle['Base'] = np.diag(df_triangle.iloc[:,1:])
df_retention = pd.DataFrame(df_triangle[['first_order', 'Base']])
df_retention.columns = ['Cohort', 'Base']

for col in df_triangle.columns[1:-1]:
    df_retention[col] = round(df_triangle[col] / df_triangle['Base'], 3)

for i in range(2, len(df_retention.columns)):
    df_retention.iloc[i-1:, i] = None

In [76]:
get_retention(get_crosstab(df_users, 'sales_sum'))

,Cohort,Base,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,...,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12
0,2014-01,14236.8950,1.0,0.016,0.000,0.103,0.003,0.000,0.008,0.064,...,0.196,0.014,0.111,0.138,0.057,0.025,0.567,0.144,0.864,0.047
1,2014-02,4294.6600,NaN,1.000,0.148,0.139,0.134,0.000,0.162,0.242,...,0.205,0.131,0.037,1.501,0.182,0.250,0.232,0.366,0.840,0.141
2,2014-03,55056.6990,NaN,NaN,1.000,0.028,0.046,0.060,0.056,0.000,...,0.070,0.056,0.087,0.104,0.057,0.055,0.087,0.045,0.158,0.111
3,2014-04,24707.1650,NaN,NaN,NaN,1.000,0.098,0.035,0.094,0.018,...,0.207,0.010,0.235,0.098,0.236,0.168,0.247,0.130,0.164,0.324
4,2014-05,18070.3410,NaN,NaN,NaN,NaN,1.000,0.093,0.103,0.036,...,0.007,0.059,0.118,0.263,0.139,0.037,0.184,0.305,0.642,0.127
5,2014-06,28735.8210,NaN,NaN,NaN,NaN,NaN,1.000,0.012,0.001,...,0.122,0.156,0.100,0.128,0.079,0.010,0.235,0.063,0.534,0.129
6,2014-07,25492.1410,NaN,NaN,NaN,NaN,NaN,NaN,1.000,0.086,...,0.039,0.265,0.143,0.175,0.035,0.195,0.094,0.312,0.232,0.199
7,2014-08,22634.6935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,...,0.042,0.039,0.116,0.123,0.129,0.108,0.076,0.168,0.189,0.355
8,2014-09,43648.8948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.050,0.113,0.028,0.085,0.214,0.084,0.388,0.420,0.098,0.099
9,2014-10,20829.9950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.216,0.030,0.036,0.213,0.127,0.087,0.160,0.413,0.278,0.171


### Transaction retention

In [77]:
get_retention(get_crosstab(df_users, 'order_counts'))

,Cohort,Base,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,...,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12
0,2014-01,32.0,1.0,0.125,0.000,0.062,0.062,0.000,0.062,0.125,...,0.188,0.125,0.156,0.125,0.031,0.094,0.281,0.312,0.406,0.219
1,2014-02,24.0,NaN,1.000,0.167,0.125,0.042,0.000,0.083,0.083,...,0.167,0.125,0.083,0.250,0.125,0.125,0.208,0.250,0.333,0.125
2,2014-03,67.0,NaN,NaN,1.000,0.075,0.045,0.119,0.104,0.000,...,0.104,0.179,0.209,0.149,0.179,0.104,0.328,0.134,0.313,0.269
3,2014-04,56.0,NaN,NaN,NaN,1.000,0.107,0.071,0.071,0.054,...,0.250,0.054,0.196,0.089,0.161,0.161,0.286,0.161,0.304,0.268
4,2014-05,57.0,NaN,NaN,NaN,NaN,1.000,0.088,0.088,0.070,...,0.053,0.088,0.070,0.246,0.158,0.105,0.333,0.158,0.368,0.175
5,2014-06,49.0,NaN,NaN,NaN,NaN,NaN,1.000,0.020,0.041,...,0.082,0.122,0.184,0.204,0.122,0.061,0.265,0.143,0.327,0.224
6,2014-07,44.0,NaN,NaN,NaN,NaN,NaN,NaN,1.000,0.136,...,0.114,0.159,0.182,0.273,0.091,0.159,0.273,0.227,0.409,0.295
7,2014-08,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,...,0.059,0.157,0.137,0.176,0.196,0.157,0.196,0.275,0.314,0.216
8,2014-09,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.123,0.123,0.110,0.164,0.192,0.123,0.301,0.137,0.233,0.178
9,2014-10,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.136,0.114,0.159,0.273,0.205,0.091,0.250,0.159,0.386,0.341
